In [1]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk import word_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk import RegexpTokenizer
from nltk import stem
from nltk.corpus import stopwords
from nltk.stem.snowball import EnglishStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.grid_search import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import fbeta_score, make_scorer
from sklearn import cross_validation
from sklearn.base import TransformerMixin
from sklearn.model_selection import learning_curve
from sklearn.metrics import roc_curve, auc

E:\python\anaconda\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
E:\python\anaconda\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


# С чем работаем?

Подготовим данные - прочтём и изучим их сбалансированность.

In [2]:
path = 'smsspamcollection/SMSSpamCollection'
mes = pandas.read_csv(path, sep='\t',names=["label", "message"])
mes['label'] = mes['label'].map({'ham': 0, 'spam': 1}).astype(int)
print(mes.groupby('label').count())

       message
label         
0         4825
1          747


Выборка, очевидно, не сбалансирована - в ~6.5 раз меньше спама чем полезных сообщений.

In [3]:
mes_train, mes_test, label_train, label_test = train_test_split(mes['message'], mes['label'], test_size=0.2)
countvec = CountVectorizer(tokenizer = word_tokenize, stop_words = "english")
train_bowed_mes = countvec.fit(mes_train)
test_bowed_mes = countvec.fit(mes_test)
train_bowed_mes = countvec.transform(mes_train)
test_bowed_mes = countvec.transform(mes_test)

# Dummy classifier

Первым делом проделаем векторизацию, иначе классификатор не сможет обработать то, что мы ему дадим, ведь у нас половина таблицы - это текст. Затем обучим классифиикатор.

In [4]:
#обучение DC
clf = DummyClassifier(strategy='most_frequent', random_state=0)
clf = clf.fit(train_bowed_mes, label_train)
print(classification_report(label_test, clf.predict(test_bowed_mes)))

             precision    recall  f1-score   support

          0       0.88      1.00      0.94       984
          1       0.00      0.00      0.00       131

avg / total       0.78      0.88      0.83      1115



E:\python\anaconda\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Мы видим, что DC не справляется с тем, чтобы определить спам. Поэтому работать мы с ним не будем

# Токенизация

Токенизация со знаками препинания. CountVectorizer()

In [5]:
#наивный Байес
naive_model = MultinomialNB()
naive_model.fit(train_bowed_mes, mes['label'])
#выдача результатов
cv_results = cross_val_score(naive_model, bowed_messages, new['label'], cv=10, scoring='accuracy')
print(cv_results.mean(), cv_results.std())
print(classification_report(naive_model1.predict(test_bowed_mes), new['label']))

ValueError: Found input variables with inconsistent numbers of samples: [4457, 5572]

Токенизация со знаками препинания. TfidfVectorizer()

In [6]:
bow_token_TIV = TfidfVectorizer()
bow_token_TIV.fit_transform(mes['message'])
bowed_messages = bow_token_TIV.transform(mes['message'])
naive_model = MultinomialNB()
naive_model.fit(bowed_messages, mes['label'])
cv_results = cross_val_score(naive_model, bowed_messages, mes['label'], cv=10, scoring='accuracy')
print(classification_report(naive_model.predict(test_bowed_mes), mes['label']))

ValueError: dimension mismatch

Токенизация без знаков препинания. CountVectorizer()

In [7]:
bow_token_P_CV = CountVectorizer(tokenizer=RegexpTokenizer(r'\w+').tokenize)
#Пытался сунуть другой токенайзер, но он отказался работать((((
bow_token_P_CV.fit_transform(mes['message'])
bowed_messages = bow_token_P_CV.transform(mes['message'])
naive_model1 = MultinomialNB()
naive_model1.fit(bowed_messages, mes['label'])
cv_results = cross_val_score(naive_model, bowed_messages, mes['label'], cv=10, scoring='accuracy')
print(classification_report(naive_model1.predict(test_bowed_mes), mes['label']))

ValueError: dimension mismatch

Токенизация без знаков препинания. TfidfVectorizer()

In [8]:
bow_token_P_TIV = TfidfVectorizer(tokenizer=RegexpTokenizer(r'\w+').tokenize)
bow_token_P_TIV.fit_transform(mes['message'])
bowed_messages = bow_token_P_TIV.transform(mes['message'])
naive_model = MultinomialNB()
naive_model.fit(bowed_messages, mes['label'])
cv_results = cross_val_score(naive_model, bowed_messages, mes['label'], cv=10, scoring='accuracy')
print(classification_report(naive_model.predict(test_bowed_mes), mes['label']))

ValueError: dimension mismatch

Токенизация со знаками препинания показала себя чуть лучше. Буквально на капельку. Можно сказать, что разницы нет вообще.
Но в обоихъ случаях TfidfVectorizer() явно хуже, чем CountVectorizer()

# Стемминг и лемматизация

Стемминг. CountVectorizer()

In [9]:
bow_stem_CV = CountVectorizer(tokenizer=RegexpTokenizer(r'\w+').tokenize)
bow_stem_CV.fit_transform(mes['message'])
bowed_messages = bow_stem_CV.transform(mes['message'])
naive_model = MultinomialNB()
naive_model.fit(bowed_messages, mes['label'])
cv_results = cross_val_score(naive_model, bowed_messages, mes['label'], cv=10, scoring='accuracy')
print(classification_report(naive_model.predict(test_bowed_mes), mes['label']))

ValueError: dimension mismatch

Стемминг. TfidfVectorizer()

In [10]:
bow_stem_TIV = TfidfVectorizer()
bow_stem_TIV.fit_transform(mes['message'])
bowed_messages = bow_stem_TIV.transform(mes['message'])
naive_model = MultinomialNB()
naive_model.fit(bowed_messages, mes['label'])
cv_results = cross_val_score(naive_model, bowed_messages, mes['label'], cv=10, scoring='accuracy')
print(cv_results.mean(), cv_results.std())
print(classification_report(naive_model.predict(test_bowed_mes), mes['label']))

0.963208815585 0.00590979736916


ValueError: dimension mismatch

Лемматизация. CountVectorizer()

In [11]:
bow_lem_CV = CountVectorizer(tokenizer=RegexpTokenizer(r'\w+').tokenize)
bow_lem_CV.fit_transform(mes['message'])
bowed_messages = bow_lem_CV.transform(mes['message'])
naive_model = MultinomialNB()
naive_model.fit(bowed_messages, mes['label'])
cv_results = cross_val_score(naive_model, bowed_messages, mes['label'], cv=10, scoring='accuracy')
print(classification_report(naive_model.predict(test_bowed_mes), mes['label']))

ValueError: dimension mismatch

Лемматизация. TfidfVectorizer()

In [12]:
bow_lem_TIV = TfidfVectorizer(tokenizer=RegexpTokenizer(r'\w+').tokenize)
bow_lem_TIV.fit_transform(mes['message'])
bowed_messages = bow_lem_TIV.transform(mes['message'])
naive_model6 = MultinomialNB()
naive_model6.fit(bowed_messages, mes['label'])
cv_results = cross_val_score(naive_model, bowed_messages, mes['label'], cv=10, scoring='accuracy')
print(classification_report(naive_model6.predict(test_bowed_mes), mes['label']))

ValueError: dimension mismatch

Между лемматизацией и стеммингом никакой разницы. При этом CountVectorizer() и TfidfVectorizer() сохраняют свои различия. 

# Стоп-слова

In [13]:
bow_SW_CV = CountVectorizer(tokenizer=RegexpTokenizer(r'\w+').tokenize, stop_words='english')
bow_SW_CV.fit_transform(mes['message'])
bowed_messages = bow_SW_CV.transform(mes['message'])
naive_model = MultinomialNB()
naive_model.fit(bowed_messages, mes['label'])
cv_results = cross_val_score(naive_model, bowed_messages, mes['label'], cv=10, scoring='accuracy')
print(classification_report(naive_model.predict(test_bowed_mes), mes['label']))

ValueError: dimension mismatch

In [14]:
bow_SW_TIV = TfidfVectorizer(tokenizer=RegexpTokenizer(r'\w+').tokenize)
bow_SW_TIV.fit_transform(mes['message'])
bowed_messages = bow_SW_TIV.transform(mes['message'])
naive_model = MultinomialNB()
naive_model.fit(bowed_messages, mes['label'])
cv_results = cross_val_score(naive_model, bowed_messages, mes['label'], cv=10, scoring='accuracy')
print(classification_report(naive_model.predict(test_bowed_mes), mes['label']))

ValueError: dimension mismatch

Впрочем, ничего нового.

Я искренне надеялся сделать побольше и сам, но увы. Я хотя бы попытался и разобрался в том, что изучил.